<a href="https://colab.research.google.com/github/kaimakov-rk/kokoko/blob/master/Copy_of_%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_%D0%BD%D0%BE%D0%B2%D0%BE%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Линейная регрессия

##Выбор и исключение переменных

Загрузите набор данных из файла ```Regressions datasets``` в соответствии с вашим вариантом (о смене датасета - индивидуально)

In [50]:
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install numpy

In [51]:
import pandas as pd

df = pd.read_csv("/content/regression.csv")

### Предварительный анализ

1. Выбрать целевую переменную Y (Y - числоая переменная для задачи регрессии)
2. С помощью EDA (см практика) отобрать признаки, которые влияют на Y (можно определить гарфически, можно числами).
3. Обработать пропуски, при необходимости почистить от выбросов (тут важное замечание - не все выбросы это плохо)
4. Обосновать выбор переменных для исследования (корреляция, различия по классам, и так далее)

P.S. Важное замечание, слишком уникальные признаки (имена людей, названия город, ID юзера или чего-то еще - не берем, так как они не влияют на целевую переменную, а если и влияет, то это случайно)

In [52]:
df.head(25)

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
5,6,2010,1,1,5,NaN,-19,-10.0,1017.0,NW,16.10,0,0
6,7,2010,1,1,6,NaN,-19,-9.0,1017.0,NW,19.23,0,0
7,8,2010,1,1,7,NaN,-19,-9.0,1017.0,NW,21.02,0,0
8,9,2010,1,1,8,NaN,-19,-9.0,1017.0,NW,24.15,0,0
9,10,2010,1,1,9,NaN,-20,-8.0,1017.0,NW,27.28,0,0


In [53]:
# 1. Target (´,,•ω•,,)♡

yname = "pm2.5"
Y = df[yname]
Y

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
43819     8.0
43820    10.0
43821    10.0
43822     8.0
43823    12.0
Name: pm2.5, Length: 43824, dtype: float64

In [54]:
# 2. Correlations ☆*:.｡.o(≧▽≦)o.｡.:*☆

correlation_matrix = df.drop(columns=[yname, 'No']).corrwith(Y, numeric_only=True)
correlation_matrix

year    -0.014690
month   -0.024069
day      0.082788
hour    -0.023116
DEWP     0.171423
TEMP    -0.090534
PRES    -0.047282
Iws     -0.247784
Is       0.019266
Ir      -0.051369
dtype: float64

Сильнее всего с целевым столбцом коррелируют столбцы DEWP и Iws (обратная корелляция). Корреляция довольно слабая, но тут уже ничего не поделать o(〒﹏〒)o

In [55]:
# 3. Обработка пропусков (・`ω´・)

df = df.dropna() # T.N.D. (total null deletion)

4. Причины, по которым были выбраны столбцы DEWP и Iws: во первых, они сильнее всего коррелируют с целевой переменной. Во вторых, точка росы влияет на количество чего-то там в воздухе, т.к. пыль собирается в росе, а не разлетается по воздуху. Скорость ветра, вероятно, влияет на скорость распространения пыли по ветру. Вероятно, также на качество воздуха влияет и направление ветра.

In [56]:
# Я оказался прав (=⌒‿‿⌒=)

temp_df = df.copy()
temp_df["cbwd"] = temp_df["cbwd"].astype('category').cat.codes
temp_df[["cbwd"]].corrwith(Y, numeric_only=True)

cbwd    0.19715
dtype: float64

### Трансформация набора данных

(OneHotEncode для категориальных переменных)

(Standard or MinMax or бининг или другое преобразование для числовых)

P.S. Целевую переменную не преобразовываем, оставляем как есть


In [62]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Няшим категориальный признак (^=◕ᴥ◕=^)

encoder = OneHotEncoder(handle_unknown='ignore')
df[["cbwd"]] = encoder.fit_transform(df[["cbwd"]])

In [58]:
df[["cbwd"]].head(5)

,cbwd
24,"(0, 2)\t1.0"
25,"(0, 2)\t1.0"
26,"(0, 2)\t1.0"
27,"(0, 2)\t1.0"
28,"(0, 2)\t1.0"


In [63]:
standard_scaler = StandardScaler()

numeric_cols = ['DEWP', 'Iws']

# Няшим простые численные признаки (=^･ｪ･^=)

df[numeric_cols] = standard_scaler.fit_transform(df[numeric_cols])

In [64]:
df[numeric_cols].describe()

,DEWP,Iws
count,4.175700e+04,4.175700e+04
mean,3.539356e-17,1.905807e-17
std,1.000012e+00,1.000012e+00
min,-2.892591e+00,-4.719510e-01
25%,-8.140912e-01,-4.449441e-01
50%,1.730880e-02,-3.727913e-01
75%,9.179922e-01,-3.943711e-02
max,1.818676e+00,1.091610e+01


### Разделение данных

Для начала, разделить данные на X и Y

Разбить данные на тестовую и обучающую выборки

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [66]:
# Невозможно приготовить нейросеть, не разбив данных ⊂(´• ω •`⊂)



'\nКод здесь\n'

### Обучение
#### 1 Регрессия
- Обучить линейную регрессию на тренировочных данных (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)
- Сдлеать .predict() для X_train, X_test
- Рассчитать метрики качества MAE, MSE, MAPE, R2
- Вывести и написать выводы о коэфициентах регрессии
- Написать формулу регрессии

Сделать выводы о переобучении модели (см метрики на тестовой выборке) сделать выводы о метриках на тестовом наборе

#### 2 Ближайшие соседи
- Обучить KNN (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor)
- Сдлеать .predict() для X_train, X_test
- Рассчитать метрики качества MAE, MSE, MAPE, R2

Сделать выводы о переобучении модели (см метрики на тестовой выборке) сделать выводы о метриках на тестовом наборе

In [60]:
"""
Код здесь
"""

'\nКод здесь\n'

##Задание для гениев*

Удачи 😈

В практике есть примеры реализации ```linear_regression``` и ```knn_regressor```, но есть нюанс... Сейчас ни linear_regression, ни knn_regressor не поддерживают сохранения состояния для последующего возможного использования.

Необходимо - реализовать два класса LinearRegression и KNNRegression (примеры обучения моделек есть). В этих классах обязатльно должны быть два метода
```
.fit(X: np.array, y: np.array) -> None - обучить модель и сохранить внутреннее состояние модели
.predict(X) -> np.array - предсказать y на основе X
```
При инициализации класса - передавать параметры модели в момент инициализации. Например:

```
model = LinearRegression(**params)

model.fit(X_train, y_train)
model.predict(X_test)
```

В качестве тестов можно использовать KNN и LinearRegression из sklearn, результат работы ваших объектов должен примерно совпадать с результатами работы объектов из sklearn

В результате работы сделать сравнение с теми же данными в sklearn

In [61]:
"""
Результат работы
"""

'\nРезультат работы\n'